- [참고 data2vec audio:사용 안하기로함]: https://towhee.io/audio-embedding/data2vec
- [참고 data2vec text]: https://huggingface.co/docs/datasets/nlp_load
- [참고 data2vec load audio*]:https://huggingface.co/docs/datasets/audio_load
- [참고 data2vec embedding]: https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecAudioModel
- [참고 handling pytorch dataset]:https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
- [참고 add data to dataset]: https://pypi.org/project/datasets/

# Embeddings(Text, Audio) Session 1 
- 각세션으로 확대시켜야 함

In [1]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob
import numpy as np


/home/arplab/project/lou/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
file_name = 'Sess01_eval.csv'
df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
# df_annotation['Emotion'], df_annotation['Valence'], df_annotation['Arousal']
file_lst = df_annotation[' .1']
file_lst

0      Sess01_script01_User002M_001
1      Sess01_script01_User002M_002
2      Sess01_script01_User002M_003
3      Sess01_script01_User002M_004
4      Sess01_script01_User001F_001
                   ...             
306    Sess01_script06_User001F_016
307    Sess01_script06_User002M_041
308    Sess01_script06_User002M_042
309    Sess01_script06_User002M_043
310    Sess01_script06_User001F_017
Name:  .1, Length: 311, dtype: object

In [2]:
session_num_lst = []
for i in range(40):
    if i <= 8:
        session_num_lst.append('Session0' + str(i+1))
    else:
        session_num_lst.append('Session' + str(i+1))
        
session_num_lst

['Session01',
 'Session02',
 'Session03',
 'Session04',
 'Session05',
 'Session06',
 'Session07',
 'Session08',
 'Session09',
 'Session10',
 'Session11',
 'Session12',
 'Session13',
 'Session14',
 'Session15',
 'Session16',
 'Session17',
 'Session18',
 'Session19',
 'Session20',
 'Session21',
 'Session22',
 'Session23',
 'Session24',
 'Session25',
 'Session26',
 'Session27',
 'Session28',
 'Session29',
 'Session30',
 'Session31',
 'Session32',
 'Session33',
 'Session34',
 'Session35',
 'Session36',
 'Session37',
 'Session38',
 'Session39',
 'Session40']

## Audio Embedding

In [52]:
wav_file_lst = []
for i in file_lst:
    wav_file_lst.append('./org_KEMDy20/Session01/' + i + '.wav')
wav_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.wav']

In [4]:
def get_file_names(file_lst):
    file_name_lst = []
    for i in file_lst:
        i = i.split('/')[-1]
        i = i.split('.')[-2]
        file_name_lst.append(i)
    return file_name_lst
    

In [5]:
import datasets

dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
dataset[0]["wav"]

{'path': './org_KEMDy20/Session01/Sess01_script01_User001F_001.wav',
 'array': array([-0.00183105, -0.00146484, -0.00140381, ..., -0.00091553,
        -0.00076294, -0.0010376 ], dtype=float32),
 'sampling_rate': 16000}

In [6]:
check_point_wav_model = 'facebook/data2vec-audio-base-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model_wav = Data2VecAudioModel.from_pretrained(check_point_wav_model)

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
dataset[0]['wav']['array']

array([-0.00183105, -0.00146484, -0.00140381, ..., -0.00091553,
       -0.00076294, -0.0010376 ], dtype=float32)

In [8]:
sampling_rate = dataset['wav'][0]['sampling_rate']
sampling_rate

16000

In [9]:
def wav_embeddings_func(wav_file_lst, processor, model):
    dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
    wav_embeddings = []
    for i in range(len(dataset)): 
        inputs = processor(dataset[i]['wav']['array'], sampling_rate = sampling_rate, return_tensors = 'pt', padding = True, max_length=400)
        with torch.no_grad():
            output = model_wav(**inputs)
        wav_embeddings.append(output['last_hidden_state'])
    return wav_embeddings

## Text Embedding


- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [10]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import re


In [51]:
# text data loading
txt_file_lst = []
for i in file_lst:
    txt_file_lst.append('./org_KEMDy20/Session01/' + i + '.txt')
txt_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.txt']

In [12]:
# 참고: https://huggingface.co/gogamza/kobart-base-v2
# 참고: https://github.com/BM-K/Sentence-Embedding-Is-All-You-Need

# model = AutoModel.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
# tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
checkpoint = 'BM-K/KoSimCSE-bert-multitask'
model_txt = AutoModel.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'


In [13]:
def text_embeddings_func(txt_file_lst, tokenizer, model):
    sentences = []

    for i in txt_file_lst:    
        f = open(i, 'r')
        line = f.readline()
        # line = re.sub('l/\n|c/|b/|n/|l/|u/', '', line)
        line = re.sub('\n', '', line)
        line = re.sub('  ', ' ', line)
        line = line.rstrip().lstrip()
        sentences.append(line)
        f.close()
    start_index = np.random.randint(1, high = len(txt_file_lst)) - 10
    print(sentences[start_index:start_index+10])
    
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    txt_embeddings, _ = model_txt(**inputs, return_dict=False)
    return txt_embeddings
    

# 데이터셋 만들기

In [14]:
# wav file embedding
wav_embeddings = wav_embeddings_func(wav_file_lst, processor, model_wav)
wav_embeddings[:3]

2023-03-27 17:14:41.527856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 17:14:42.452983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda-11/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-03-27 17:14:42.453067: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda

[tensor([[[-1.0521,  0.0236, -0.0885,  ...,  0.0592, -0.0703,  0.1332],
          [-1.0521,  0.0236, -0.0885,  ...,  0.0592, -0.0703,  0.1332],
          [-1.0521,  0.0236, -0.0885,  ...,  0.0592, -0.0703,  0.1332],
          ...,
          [-1.1249,  0.0886, -0.0205,  ..., -0.1581, -0.2275,  0.0239],
          [-1.1264,  0.0884, -0.0214,  ..., -0.1584, -0.2291,  0.0230],
          [-1.1284,  0.0942, -0.0229,  ..., -0.1598, -0.2316,  0.0242]]]),
 tensor([[[-1.2183,  0.2462, -0.3427,  ...,  0.0913,  0.0475,  0.2768],
          [-1.2183,  0.2462, -0.3427,  ...,  0.0913,  0.0475,  0.2768],
          [-1.2183,  0.2462, -0.3427,  ...,  0.0913,  0.0475,  0.2768],
          ...,
          [-1.1338,  0.1411, -0.0790,  ..., -0.1295, -0.0888,  0.0974],
          [-1.1310,  0.1384, -0.0780,  ..., -0.1281, -0.0885,  0.0973],
          [-1.1288,  0.1362, -0.0776,  ..., -0.1268, -0.0862,  0.0965]]]),
 tensor([[[-0.6320,  0.0581, -0.4626,  ...,  0.1816, -0.1539,  0.4442],
          [-0.6320,  0.0581,

In [15]:
# txt file embedding
txt_embeddings = text_embeddings_func(txt_file_lst,tokenizer, model_txt)
txt_embeddings[:3]

['c/ 내가 그냥 그 일상적인 전화로 대화하거나 하는데.', '그래갖고 되게 특히 친구랑 전화하는데 웃겼던 적도 있었었어.', '어 막 한참 웃으면서 얘기하고 있는데 갑자기 막 노래 노래 막 팝송 나오고 l/ 어.', '어어 l/ b/ 되게 BGM처럼 막 깔아줬던 웃겼던 적도 있었었어.', 'b/ u/ 그것 말고 또 뭐가 있었을까?', '되게 웃겼던 적은 막 인식을 진짜 잘못해서 하는 경우가 대부분인 거 같애.', 'c/ 그 영상에서 봤다시피.', '우리 집은 되게 자유로워 가지고 솔직히 아빠가 이게 어때라고 하면 본인 생 그니까 가족들 생각이랑 안 맞으면 그냥 자기 주관대로 가거든, 다들.', '그래서 좀 자유로워서 아빠가 b/ 가부장적인 면이 없지 않아 있다고는 생각을 하는데 그래도 우리가 그냥 안 듣다 보니까 그런 게 없 없는 듯이 느껴지는 거 같애.', '가부장적이다라는 거?']


tensor([[[-0.7713,  0.3540, -0.1561,  ..., -0.8406, -0.5942,  0.7619],
         [-0.0500, -1.1157,  0.3815,  ..., -0.3483,  0.5291,  0.6400],
         [-1.4624, -0.6926, -0.6904,  ..., -0.1558,  0.2706,  1.2804],
         ...,
         [-0.4228,  0.6478,  0.0635,  ..., -0.9161,  0.1675,  0.7280],
         [-0.6350,  0.4763, -0.0086,  ..., -0.8950,  0.3458,  0.6184],
         [-0.7193, -0.0301, -0.1823,  ..., -0.5610,  0.4327,  0.3804]],

        [[ 0.5551, -0.4455, -0.0732,  ...,  1.3076,  0.8142,  1.6034],
         [ 0.0394, -0.5077,  0.7435,  ...,  0.6888,  0.9015,  1.2672],
         [-0.8009, -0.2002, -0.3190,  ...,  1.1466,  0.2687,  1.8889],
         ...,
         [ 0.8747, -0.5071, -0.1666,  ...,  1.4158,  1.1544,  1.0865],
         [ 0.7265, -0.7809, -0.0251,  ...,  1.1178,  1.1242,  1.1936],
         [ 0.6910, -0.9297, -0.2339,  ...,  0.6449,  1.0709,  1.6147]],

        [[-0.4392,  0.2658,  0.3925,  ...,  0.9807,  1.5450,  1.4830],
         [-0.5285,  0.5159,  0.2705,  ...,  0

In [53]:
file_name_lst = get_file_names(wav_file_lst)
file_name_lst[:5]

['Sess01_script01_User002M_001',
 'Sess01_script01_User002M_002',
 'Sess01_script01_User002M_003',
 'Sess01_script01_User002M_004',
 'Sess01_script01_User001F_001']

In [57]:
# annotation data 가져오기
df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
df_annotation.Arousal[:3], df_annotation.Emotion[:3], df_annotation.Valence[:3]

(0    2.9
 1    2.9
 2    3.0
 Name: Arousal, dtype: float64,
 0    neutral
 1    neutral
 2    neutral
 Name: Emotion, dtype: object,
 0    3.4
 1    3.1
 2    3.1
 Name: Valence, dtype: float64)

In [17]:
print(len(txt_embeddings), len(wav_embeddings), len(file_name_lst))

311 311 311


In [64]:
def build_dataset(file_name_lst, 
                  text_embeddings, 
                  wav_embeddings, 
                  label_emotion,
                  label_arousal,
                  label_valence, 
                  session_name = 'Session01'):
    dataset = {session_name: 
        {'file_names': file_name_lst, 
         'text_embeddings': text_embeddings, 
         'wav_embeddings':wav_embeddings,
         'Emotion':label_emotion,
         'Arousal':label_arousal,
         'Valence':label_valence}}
    return dataset

In [68]:
dataset = build_dataset(file_name_lst, 
                txt_embeddings, 
                wav_embeddings,  
                df_annotation.Arousal, 
                df_annotation.Emotion, 
                df_annotation.Valence,
                session_name = 'Session01')
dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])

In [72]:
# length test
for i in dataset['Session01'].keys():
    print(len(dataset['Session01'][i]))


311
311
311
311
311
311


In [73]:
import pickle

# save data set
with open('./model/data/lou_dataset.pkl', 'wb')as f:
    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

In [74]:
with open('./model/data/lou_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])